## Project 1 Exercise 2 - Regression, Bike sharing Dataset
##### <i>By Shanaaz Kamal</i>
##### <i>Module: 212BUS-212A-1 : Analyzing Big Data II</i>
##### I'll be using the bike sharing dataset to predict the count of casual, registered and both types of riders

#### Import required packages

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from dmba import regressionSummary
from dmba import adjusted_r2_score, AIC_score, BIC_score
from dmba import forward_selection

no display found. Using non-interactive Agg backend


#### Load the dataset

In [3]:
day_df = pd.read_csv("day.csv")
hour_df = pd.read_csv("hour.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'day.csv'

In [ ]:
day_df.head(6)

In [ ]:
day_df.shape

In [ ]:
hour_df.head(6)

In [ ]:
hour_df.shape

#### Data pre-processing (Clean and Modify data)

In [ ]:
## Check for duplicate data, null records, number of rows and number of columns.
print("For the day_df dataset,")
duplicate_day = day_df.duplicated()
print("There are",(duplicate_day.sum()),"number of duplicated records")
print("There are",sum(day_df.isnull().sum()),"number of null records")
print("There are",day_df.shape[0],"number of rows and",day_df.shape[1],"number of columns")

In [ ]:
## Check for duplicate data, null records, number of rows and number of columns.
print("For the hour_df dataset,")
duplicate_hour= hour_df.duplicated()
print("There are",(duplicate_hour.sum()),"number of duplicated records")
print("There are",sum(hour_df.isnull().sum()),"number of null records")
print("There are",hour_df.shape[0],"number of rows and",hour_df.shape[1],"number of columns")

In [ ]:
## check for number of unique values
day_df.nunique()

In [ ]:
## check for number of unique values
hour_df.nunique()

In [ ]:
## To view column type
day_df.info() 

In [ ]:
hour_df.info()

In [ ]:
## View the day_df information, especially for the numerical variables [temp,atemp,hum,windspeed]
day_df.describe()

In [ ]:
## View the hour_df information, especially for the numerical variables [temp,atemp,hum,windspeed]
hour_df.describe()

In [ ]:
## Categorical variables for hour_df dataset
categorical_vars_hour = ['yr','holiday','workingday','season','mnth','hr','weekday','weathersit']

## Categorical variables for day_df dataset
categorical_vars = ['season','yr','mnth','holiday','weekday','workingday','weathersit']

## Store the numerical variable into a list
numerical_vars =['temp','atemp','hum','windspeed']

#### Exploratory Data Analysis Visualizations

In [ ]:
## Visualize the categorical variables
sns.set_theme(style="darkgrid")

fig = plt.figure(figsize=(50,30))

## Reduce the space between the plots
gs = fig.add_gridspec(5,5)
gs.update(wspace=0.2, hspace=0.15)
fig = plt.figure(figsize=(50,30))

ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[0,2])
ax3 = fig.add_subplot(gs[1,0])
ax4 = fig.add_subplot(gs[1,1])
ax5 = fig.add_subplot(gs[1,2])
ax6 = fig.add_subplot(gs[2,0])
ax7 = fig.add_subplot(gs[2,1])

color_palette = "magma"

ax0.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.countplot(ax=ax0,data=day_df,x='season',palette=color_palette)
ax0.set_xlabel("Season",fontsize=18)
ax0.set_ylabel("count",fontsize=18)

ax1.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.countplot(ax=ax1,data=day_df,x='mnth',palette=color_palette)
ax1.set_xlabel("Month",fontsize=18)
ax1.set_ylabel("")

ax2.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.countplot(ax=ax2,data=day_df,x='weekday',palette=color_palette)
ax2.set_xlabel("Day", fontsize=18)
ax2.set_ylabel("count", fontsize=18)

ax3.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.countplot(ax=ax3,data=day_df,x='weathersit',palette=color_palette)
ax3.set_xlabel("Weather Situation", fontsize=18)
ax3.set_ylabel("")

ax4.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.countplot(ax=ax4,data=hour_df,x='hr',palette=color_palette)
ax4.set_xlabel("Hour", fontsize=18)
ax4.set_ylabel("")

ax5.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.countplot(ax=ax5,data=day_df,x='yr',palette=color_palette)
ax5.set_xlabel("Year", fontsize=18)
ax5.set_ylabel("")

ax6.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.countplot(ax=ax6,data=day_df,x='holiday',palette=color_palette)
ax6.set_xlabel("Holiday", fontsize=18)
ax6.set_ylabel("")

ax7.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.countplot(ax=ax7,data=day_df,x='workingday',palette=color_palette)
ax7.set_xlabel("Working Day", fontsize=18)
ax7.set_ylabel("")


ax0.grid(True, which='major')
ax1.grid(True, which='major')
ax2.grid(True, which='major')
ax3.grid(True, which='major')
ax4.grid(True, which='major')
ax5.grid(True, which='major')
ax6.grid(True, which='major')
ax7.grid(True, which='major')


In [ ]:
## Visualize the numerical/continuous variables (not including the indicator/dummy variables)

sns.set_theme(style="darkgrid")

fig = plt.figure(figsize=(40,30))
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[1,1])

## Reduce the space between the plots
gs = fig.add_gridspec(4,4)
gs.update(wspace=0.2, hspace=0.15)

color_palette = "magma"

ax0.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax0,data=day_df,y='temp',palette=color_palette, width=0.3)
ax0.set_xlabel("")
ax0.set_ylabel("Normalized Temperature",fontsize=18)

ax1.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax1,data=day_df,y='atemp',palette=color_palette, width=0.3)
ax1.set_xlabel("")
ax1.set_ylabel("Feeling Temperature",fontsize=18)

ax2.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax2,data=day_df,y='hum',palette=color_palette, width=0.3)
ax2.set_xlabel("")
ax2.set_ylabel("Humidity",fontsize=18)

ax3.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax3,data=day_df,y='windspeed',palette=color_palette, width=0.3)
ax3.set_xlabel("")
ax3.set_ylabel("Wind Speed",fontsize=18)


ax0.grid(True, which='major')
ax1.grid(True, which='major')
ax2.grid(True, which='major')
ax3.grid(True, which='major')

In [ ]:
## Visualize the correlation matrix between the variables in day_df dataset
corr = day_df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
## Visualize the correlation matrix between the variables in hour_df dataset
corr = hour_df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
## Visualize relationship between the target variables ('casual','registered',cnt) and predictors/features for day_df
y_vars = ["casual","registered","cnt"]
sns.pairplot(day_df,hue='yr',palette = ["#8000ff","#da8829"],y_vars=y_vars,height=3)

In [ ]:
## Visualize relationship between the target variables ('casual','registered',cnt) and predictors/features for hour_df
y_vars = ["casual","registered","cnt"]
sns.pairplot(hour_df,hue='yr',palette = ["#8000ff","#da8829"],y_vars=y_vars,height=3)

In [ ]:
## Visualize the relationship between the numerical variables and target variables in day_df dataset

x_vars = ["temp","atemp","hum","windspeed"]
y_vars = ["casual","registered","cnt"]
g = sns.PairGrid(day_df, hue="yr", x_vars=x_vars, y_vars=y_vars,palette="magma",height=6,aspect=0.9)
g.map_diag(sns.histplot) 
g.map_offdiag(sns.scatterplot)
g.add_legend(title='Year', fontsize= '12')
plt.figure(figsize=(10, 10))

In [ ]:
## Visualize the relationship between the numerical variables and target variables in hour_df
 
x_vars = ["temp","atemp","hum","windspeed"]
y_vars = ["casual","registered","cnt"]
g = sns.PairGrid(hour_df, hue="yr", x_vars=x_vars, y_vars=y_vars,palette="magma",height=6,aspect=0.9)
g.map_diag(sns.histplot) 
g.map_offdiag(sns.scatterplot)
g.add_legend(title='Year', fontsize= '12')
ax0.set_xlabel("")
ax0.set_ylabel("Normalized Temperature",fontsize=18)

plt.figure(figsize=(10, 10))

### Data Observations ###
* No duplicates/null in the data
* There more number of riders when the weather situation = 1(Clear, Few Clouds, Partly Cloudy)
* There are more registered riders in 2012 than 2011
* Registered riders use the bike when temperatures, humidity and windspeed are more optimal, in comparison to casual riders
* Casual riders do not use the bike when windspeed is high
* Based on the correlation matrix, there is not much collinearity among the variables in both the day and hour datasets
* From the pairgrid, we can tell that there are more casual/registered/both riders for higher humidity
* Based on the pairplot, there seems to be better distribution of the data for the day_df dataset

##### I will proceed in using the day dataset to predict the target variables casual, registered and both type users.

#### Split the dataset before scaling

In [ ]:
train_df, test_df = train_test_split(day_df, test_size=0.2)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
train_df.shape, test_df.shape

#### Scale and Transform the dataset for modelling

In [ ]:
train_df.describe()

In [ ]:
## Initialized the MinMaxScaler to scale the numericals variabls to 0 and 1
scaler = MinMaxScaler()
train_transformed_df=pd.DataFrame(scaler.fit_transform(train_df[numerical_vars]))

In [ ]:
## Added the column names
train_transformed_df.columns=['temp','atemp','hum','windspeed']

In [ ]:
## Verified shape
train_transformed_df.shape

In [ ]:
## Obtain the dummy variables for the categorical data
def get_one_hot_encoded_col(train_df, test_df, input_col):
    enc = OneHotEncoder(handle_unknown = 'ignore')
    enc.fit(np.array(train_df[input_col]).reshape(-1, 1))

    encoded_train = enc.transform(np.array(train_df[input_col]).reshape(-1, 1)).toarray()
    encoded_test = enc.transform(np.array(test_df[input_col]).reshape(-1, 1)).toarray()
    
    encoded_cols = ['{}_{}'.format(input_col, i) for i in range(encoded_train.shape[1])]
    
    return encoded_train, encoded_test, encoded_cols

In [ ]:
oh_season_train, oh_season_test, oh_season_cols = get_one_hot_encoded_col(train_df, test_df, 'season')
oh_yr_train, oh_yr_test, oh_yr_cols = get_one_hot_encoded_col(train_df, test_df, 'yr')
oh_mnth_train, oh_mnth_test, oh_mnth_cols = get_one_hot_encoded_col(train_df, test_df, 'mnth')
oh_holiday_train, oh_holiday_test, oh_holiday_cols = get_one_hot_encoded_col(train_df, test_df, 'holiday')
oh_weekday_train, oh_weekday_test, oh_weekday_cols = get_one_hot_encoded_col(train_df, test_df, 'weekday')
oh_workdingday_train, oh_workdingday_test, oh_workingday_cols = get_one_hot_encoded_col(train_df, test_df, 'workingday')
oh_weathersit_train, oh_weathersit_test, oh_weathersit_cols = get_one_hot_encoded_col(train_df, test_df, 'weathersit')

In [ ]:
dummy_categorical_train_df = pd.DataFrame(np.concatenate((oh_season_train, oh_yr_train, 
                                                    oh_mnth_train, oh_holiday_train, oh_weekday_train, oh_workdingday_train, 
                                                    oh_weathersit_train), axis=1))

In [ ]:
train_transformed_df = pd.concat([train_transformed_df,dummy_categorical_train_df],axis=1)

In [ ]:
## Add columns to the converted dataframe
train_transformed_df.columns = ['temp', 'atemp', 'hum', 'windspeed'] + oh_season_cols + oh_yr_cols + oh_mnth_cols + oh_holiday_cols + oh_weekday_cols + oh_workingday_cols + oh_weathersit_cols

## Print column names
train_transformed_df.shape

In [ ]:
train_transformed_df.head(6)

In [ ]:
## Transform the test dataset
test_transformed_df=pd.DataFrame(scaler.transform(test_df[numerical_vars]))
test_transformed_df.columns=['temp','atemp','hum','windspeed']
test_transformed_df.shape

In [ ]:
dummy_categorical_test_df = pd.DataFrame(np.concatenate((oh_season_test, oh_yr_test, 
                                                    oh_mnth_test, oh_holiday_test, oh_weekday_test, oh_workdingday_test, 
                                                    oh_weathersit_test), axis=1))

In [ ]:
test_transformed_df = pd.concat([test_transformed_df,dummy_categorical_test_df],axis=1)

In [ ]:
## Add columns to the converted dataframe
test_transformed_df.columns = ['temp', 'atemp', 'hum', 'windspeed'] + oh_season_cols + oh_yr_cols + oh_mnth_cols + oh_holiday_cols + oh_weekday_cols + oh_workingday_cols + oh_weathersit_cols

## Print column names
test_transformed_df.shape

In [ ]:
test_transformed_df.head(6)

#### Train the model

In [ ]:
## Initialize linear regession model to predict count of casual riders
casual_lr = LinearRegression()

## Fit the training data to the LinearRegression model
casual_lr.fit(train_transformed_df, train_df['casual'])

casual_lr.score(train_transformed_df, train_df['casual'])


In [ ]:
## Initialize linear regession model
registered_lr = LinearRegression()

## Fit the training data to the LinearRegression model
registered_lr.fit(train_transformed_df, train_df['registered'])

In [ ]:
## Initialize linear regession model
cnt_lr = LinearRegression()

## Fit the training data to the LinearRegression model
cnt_lr.fit(train_transformed_df, train_df['cnt'])

#### Test the predictions

In [ ]:
test_predictions_casual = casual_lr.predict(test_transformed_df)
regressionSummary(test_df['casual'],test_predictions_casual)
print('\n\t\tAdjusted R2 : ', adjusted_r2_score(test_df['casual'], test_predictions_casual, casual_lr))
print('\t\tAIC : ', AIC_score(test_df['casual'], test_predictions_casual, casual_lr))
print('\t\tBIC : ', BIC_score(test_df['casual'], test_predictions_casual, casual_lr))

In [ ]:
test_predictions_registered = registered_lr.predict(test_transformed_df)
regressionSummary(test_df['registered'],test_predictions_registered)
print('\n\t\tAdjusted R2 : ', adjusted_r2_score(test_df['registered'], test_predictions_registered, registered_lr))
print('\t\tAIC : ', AIC_score(test_df['registered'], test_predictions_registered, registered_lr))
print('\t\tBIC : ', BIC_score(test_df['registered'], test_predictions_registered, registered_lr))

In [ ]:
test_predictions_both = cnt_lr.predict(test_transformed_df)
regressionSummary(test_df['cnt'],test_predictions_both)
print('\n\t\tAdjusted R2 : ', adjusted_r2_score(test_df['cnt'], test_predictions_both, cnt_lr))
print('\t\tAIC : ', AIC_score(test_df['cnt'], test_predictions_both, cnt_lr))
print('\t\tBIC : ', BIC_score(test_df['cnt'], test_predictions_both, cnt_lr))

#### Feature selection

##### <b>Forward selection for target variable, count of casual riders</b>

In [ ]:
## Forward selection
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_transformed_df[variables], train_df['casual'])
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score( train_df['casual'], [ train_df['casual'].mean()] * len( train_df['casual']), model, df=1)
    return AIC_score( train_df['casual'], model.predict(train_transformed_df[variables]), model)

best_model_casual, best_variables_casual = forward_selection(train_transformed_df.columns, train_model, score_model, verbose=True)
print(best_variables_casual)

##### <b>Forward selection for target variable, count of registered riders</b>

In [ ]:
## Forward selection
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_transformed_df[variables], train_df['registered'])
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score( train_df['registered'], [ train_df['registered'].mean()] * len( train_df['registered']), model, df=1)
    return AIC_score( train_df['registered'], model.predict(train_transformed_df[variables]), model)

best_model_registered, best_variables_registered = forward_selection(train_transformed_df.columns, train_model, score_model, verbose=True)
print(best_variables_registered)

##### <b>Forward selection for target variable, count of both casual and registered riders</b>

In [ ]:
## Forward selection
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_transformed_df[variables], train_df['cnt'])
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score( train_df['cnt'], [ train_df['cnt'].mean()] * len( train_df['cnt']), model, df=1)
    return AIC_score( train_df['cnt'], model.predict(train_transformed_df[variables]), model)

best_model_cnt, best_variables_cnt = forward_selection(train_transformed_df.columns, train_model, score_model, verbose=True)
print(best_variables_cnt)

#### Performance of models for casual, registered and both types of riders

In [ ]:
## Verify the performance of the test data using the best model and variables
print("Performance of Casual riders model")
print("\nBefore Feature Selection")
regressionSummary(test_df['casual'],test_predictions_casual)
print('\n\t\tAdjusted R2 : ', adjusted_r2_score(test_df['casual'], test_predictions_casual, casual_lr))
print('\t\tAIC : ', AIC_score(test_df['casual'], test_predictions_casual, casual_lr))
print('\t\tBIC : ', BIC_score(test_df['casual'], test_predictions_casual, casual_lr))
print("\nAfter Feature Selection")
regressionSummary(test_df['casual'], best_model_casual.predict(test_transformed_df[best_variables_casual]))
print('\n\t\tAdjusted R2 : ', adjusted_r2_score(test_df['casual'], best_model_casual.predict(test_transformed_df[best_variables_casual]), best_model_casual))
print('\t\tAIC : ', AIC_score(test_df['casual'], best_model_casual.predict(test_transformed_df[best_variables_casual]), best_model_casual))
print('\t\tBIC : ', BIC_score(test_df['casual'], best_model_casual.predict(test_transformed_df[best_variables_casual]), best_model_casual))

In [ ]:
## Verify the performance of the test data using the best model and variables
print("Performance of Registered riders model")
print("\nBefore Feature Selection")
regressionSummary(test_df['registered'],test_predictions_registered)
print('\n\t\tAdjusted R2 : ', adjusted_r2_score(test_df['registered'], test_predictions_registered, registered_lr))
print('\t\tAIC : ', AIC_score(test_df['registered'], test_predictions_registered, registered_lr))
print('\t\tBIC : ', BIC_score(test_df['registered'], test_predictions_registered, registered_lr))
print("\nAfter Feature Selection")
regressionSummary(test_df['registered'], best_model_registered.predict(test_transformed_df[best_variables_registered]))
print('\n\t\tAdjusted R2 : ', adjusted_r2_score(test_df['registered'], best_model_registered.predict(test_transformed_df[best_variables_registered]), best_model_registered))
print('\t\tAIC : ', AIC_score(test_df['registered'], best_model_registered.predict(test_transformed_df[best_variables_registered]), best_model_registered))
print('\t\tBIC : ', BIC_score(test_df['registered'], best_model_registered.predict(test_transformed_df[best_variables_registered]), best_model_registered))

In [ ]:
## Verify the performance of the test data using the best model and variables
print("Performance of both Casual and Registered riders model")
print("\nBefore Feature Selection")
regressionSummary(test_df['cnt'],test_predictions_both)
print('\n\t\tAdjusted R2 : ', adjusted_r2_score(test_df['cnt'], test_predictions_both, cnt_lr))
print('\t\tAIC : ', AIC_score(test_df['cnt'], test_predictions_both, cnt_lr))
print('\t\tBIC : ', BIC_score(test_df['cnt'], test_predictions_both, cnt_lr))
print("\nAfter Feature Selection")
regressionSummary(test_df['cnt'], best_model_cnt.predict(test_transformed_df[best_variables_cnt]))
print('\n\t\tAdjusted R2 : ', adjusted_r2_score(test_df['cnt'], best_model_cnt.predict(test_transformed_df[best_variables_cnt]), best_model_cnt))
print('\t\tAIC : ', AIC_score(test_df['cnt'], best_model_cnt.predict(test_transformed_df[best_variables_cnt]), best_model_cnt))
print('\t\tBIC : ', BIC_score(test_df['cnt'], best_model_cnt.predict(test_transformed_df[best_variables_cnt]), best_model_cnt))

#### Data Observations
* In general, the models to predict the features (casual, registered and both) performed better after feature selection
* This is substantiated by the higher adjusted R-squared values and the lower RMSE values